# Gradient-Conjugate

In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as sopt
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt

In [2]:
def funcx(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(funcx([x[0]+delta,x[1]])- funcx([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(funcx([x[0],x[1]+delta])- funcx([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = funcx(x+alpha1*search);
    falpha2 = funcx(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = funcx(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = funcx(x+alpha1*search);

        if np.abs(funcx(x+alpha1*search)- funcx(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [5]:
def hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= ( funcx([x[0]+delta,x[1]])  - 2*funcx(x) + funcx([x[0]-delta,x[1]]) )/ delta**2;
    H[1,1]= ( funcx([x[0],x[1]+delta])  - 2*funcx(x) + funcx([x[0],x[1]-delta]) )/ delta**2; 
    H[0,1]= ( funcx([x[0]+delta,x[1]+delta]) - funcx([x[0]+delta,x[1]-delta]) - funcx([x[0]-delta,x[1]+delta]) + funcx([x[0]-delta,x[1]-delta]) )/ (4*(delta**2));
    H[1,0]=H[0,1]
    return H

In [11]:
delta=1e-3; 
ep1=1e-3; 
xi = [-1,1];
x = xi;
fx_prev=funcx(x)
print('Initial function value = {0:.3f} '.format(fx_prev))
print(' No.\tx-vector\tf(x)\tDeriv ')
print('------------------------------------------')
for i in range(30):
    if i==0:
        dire_prev=gradient(x,delta)
        si_prev = -dire_prev
        alpha,fx_prev = golden(x,si_prev,xi,ep1);
        
        if la.norm(dire_prev)<ep1:
            break
        x = x +  alpha*si_prev 
    else:
        dire=gradient(x,delta)
        si = - dire +((la.norm(dire)**2)/(la.norm(dire_prev)**2))*si_prev;
        alpha,fx_curr = golden(x,si,xi,ep1);
        if abs(fx_curr-fx_prev)<ep1 or la.norm(dire)<ep1:
            break;
        dire_prev=dire
        si_prev=si
        x = x +  (alpha*si).transpose() 
        fx_prev=funcx(x)
    print('{0}\t[{1:.3f},{2:.3f}]\t{3:.3f}\t{4:.3f}'.format(i, x[0], x[1],fx_curr,la.norm(dire)))
print('------------------------------------------')

Initial function value = 270.294 
 No.	x-vector	f(x)	Deriv 
------------------------------------------
0	[-0.389,-0.009]	-7.353	0.224
1	[0.030,0.199]	-1.026	33.677
2	[0.438,0.101]	-6.301	39.919
3	[0.502,-0.001]	-7.353	26.034
------------------------------------------
